# Imports & Data

**Imports**

In [ ]:
# External Libraries
import os
import matplotlib.pyplot as plt
import numpy as np
from time import time
# My Code
from gmres import GMRES
from msr_reader import msr_reader

**Read Matrices**

In [ ]:
# Paths
directory = os.getcwd() + "\\matrices\\"
matrix_files = ["cg_matrix_msr_1.txt", # 0
            "cg_matrix_msr_2.txt", # 1
            "gmres_matrix_msr_1.txt", # 2
            "msr_test_non_symmetric.txt", # 3
            "msr_test_symmetric.txt"] # 4

# Read each matrix
matrices = []
for file in matrix_files:
    path = directory + f"{file}"
    matrices.append(msr_reader(path))

# GMRES

**Common Settings**

In [ ]:
# Types of preconditioning applied
preconditioners = [None,
                   "jacobi",
                   "gauss_seidel",
                   "ilu0"]

# Further Specifications
A = matrices[2]
n = A.shape[0]
x = np.ones(n)
x0 = np.zeros(n)
b = A.dot(x)
tol = 1e-8

# GMRES

**Task 1: full GMRES**

For the full GMRES method, with and without preconditioning, plot the relative residual
against iteration index on a semi-log scale.

In [ ]:
# Max number of Krylov vectors
ms = [600]

# Timing
durations = []
last_time = time()
# Loop
results = []
for m in ms:
    for preconditioner in preconditioners:
        x_approx, errors = GMRES(A, b, x0, m, tol, preconditioner, max_iterations=1000)
        results.append(errors)
        # Status message
        durations.append(round(time() - last_time, 3))
        last_time = time()
        n_conv = len(errors)
        print(f"Test 1: sek={durations[-1]}, m={m}, prec={preconditioner} iterations={n_conv} ok")

**Results**

Test 1: sek=7.166, m=600, prec=None iterations=562 ok

Test 1: sek=3.801, m=600, prec=jacobi iterations=44 ok

Test 1: sek=3.303, m=600, prec=gauss_seidel iterations=26 ok

Test 1: sek=72.846, m=600, prec=ilu0 iterations=17 ok


In [ ]:
# Print & Plot results
plt.figure(figsize=(10, 4))

for i, errors in enumerate(results):
    # Print number of iterations for convergence
    n_conv = len(errors)
    iterations = range(n_conv)
    plt.plot(iterations, np.abs(errors), label=f'{preconditioners[i]}')

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 600])
plt.ylim([1e-8, 1])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(650/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
plt.title(f'Convergence of full GMRES with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_full_gmres.png"
plt.savefig(filename, dpi=300)
plt.show()

**Task 2: GMRES(m)**

For GMRES(m), test and compare the following restart parameters m = 10, 50, 200 with
the runtime required by the full GMRES.

In [ ]:
# Max number of Krylov vectors
ms = [10, 50, 200]

# Timing
durations = []
last_time = time()

# Loop
results = []
for m in ms:
    for preconditioner in preconditioners:
        x_approx, errors = GMRES(A, b, x0, m, tol, preconditioner, max_iterations=1000)
        # Status message
        durations.append(round(time() - last_time, 3))
        last_time = time()
        n_conv = len(errors)
        if abs(errors[-1]) > 1:
            print(f"Test 2: sek={durations[-1]}, m={m}, prec={preconditioner} iterations={n_conv} diverged")
        else:
            results.append(errors)
            print(f"Test 2: sek={durations[-1]}, m={m}, prec={preconditioner} iterations={n_conv} ok")


**Results**

Test 2: sek=6.655, m=10, prec=None iterations=1101 ok

Test 2: sek=2.756, m=10, prec=jacobi iterations=12 diverged

Test 2: sek=2.931, m=10, prec=gauss_seidel iterations=12 diverged

Test 2: sek=47.913, m=10, prec=ilu0 iterations=12 diverged

Test 2: sek=6.275, m=50, prec=None iterations=1021 ok

Test 2: sek=2.796, m=50, prec=jacobi iterations=44 ok

Test 2: sek=2.577, m=50, prec=gauss_seidel iterations=26 ok

Test 2: sek=71.908, m=50, prec=ilu0 iterations=17 ok

Test 2: sek=6.632, m=200, prec=None iterations=835 ok

Test 2: sek=2.427, m=200, prec=jacobi iterations=44 ok

Test 2: sek=2.715, m=200, prec=gauss_seidel iterations=26 ok

Test 2: sek=71.536, m=200, prec=ilu0 iterations=17 ok


**m=10**

In [ ]:
# Print & Plot results
plt.figure(figsize=(10, 4))
plt.cla()

# Data
errors = results[0]
n_conv = len(errors)
iterations = range(n_conv)
plt.plot(iterations, np.abs(errors), label=f"no preconditioner")

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 600])
plt.ylim([1e-8, 1])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(650/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
plt.title(f'Convergence of GMRES(10) with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_gmres_10.png"
plt.savefig(filename, dpi=300)
plt.show()

**m=50**

In [ ]:
# Print & Plot results
plt.figure(figsize=(10, 4))
plt.cla()

# Data
for i in range(4):
    errors = results[1+i]
    n_conv = len(errors)
    iterations = range(n_conv)
    plt.plot(iterations, np.abs(errors), label=f"{preconditioners[i]}")

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 600])
plt.ylim([1e-8, 1])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(650/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
plt.title(f'Convergence of GMRES(50) with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_gmres_50.png"
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
# Print & Plot results
plt.figure(figsize=(10, 4))
plt.cla()

# Data
for i in range(4):
    errors = results[5+i]
    n_conv = len(errors)
    iterations = range(n_conv)
    plt.plot(iterations, np.abs(errors), label=f"{preconditioners[i]}")

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 600])
plt.ylim([1e-8, 1])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(650/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel('Residual Norm (log scale)')
plt.title(f'Convergence of GMRES(200) with and without preconditioning')
plt.legend()

# Save and Plot
filename = "conv_gmres_200.png"
plt.savefig(filename, dpi=300)
plt.show()

**Task 3: Orthogonality**

For full GMRES (without preconditioning): check the orthogonality of the Krylov vectors.

**Hints**: Plot the computed values of v1 · vk against k on a semi-log scale.

In [ ]:
# Max number of Krylov vectors
m = 600

# Timing
durations = []
last_time = time()
# Loop
results = []
preconditioner = None
x_approx, dot_products = GMRES(A, b, x0, m, tol, preconditioner, max_iterations=1000, orthogonality=True)
# Status message
durations.append(round(time() - last_time, 3))
last_time = time()
n_conv = len(dot_products)
print(f"Test 1: sek={durations[-1]}, m=600, prec={preconditioner} iterations={n_conv} ok")


In [ ]:
# Print & Plot results
plt.figure(figsize=(10, 2))
plt.cla()

# Data
n_conv = len(dot_products)
iterations = range(n_conv)
plt.plot(iterations, np.abs(dot_products), label=f"no preconditioner")

# Grid and Scale
# plt.yscale('log')
plt.grid(True)
# plt.xlim([0, 600])
plt.ylim([-1, 1])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(650/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel(r'Dot product $v_0\cdot v_i$')
plt.title(r'Orthogonality check for GMRES without preconditioning: $v_0\cdot v_i$')
plt.legend()

# Save and Plot
filename = "conv_gmres_orthogonality.png"
plt.savefig(filename, dpi=300)
plt.show()

# Conjugate Gradient Method

In [ ]:
import os
from msr_reader import msr_reader
from conjugate_gradient import conjugate_gradient

# Paths
directory = os.getcwd() + "\\matrices\\"
matrix_files = ["cg_matrix_msr_1.txt", # 0
                "cg_matrix_msr_2.txt", # 1
                "gmres_matrix_msr_1.txt", # 2
                "msr_test_non_symmetric.txt", # 3
                "msr_test_symmetric.txt"] # 4

# Read each matrix
matrices = []
for file in matrix_files:
    path = directory + f"{file}"
    matrices.append(msr_reader(path))
print("matrices read")

In [ ]:
# Create Initial Conditions
A = matrices[1]
n = A.shape[0]
x = np.ones(n)
x0 = np.zeros(n)
b = A.dot(x)
tol = 1e-8
m = 1000

# Solve
x_approx, a_norm_errors, residuals = conjugate_gradient(A, b, x0, m, tol)
np.round(x_approx, 2)[-10:]

# calculate r_norms
r_norms = []
for r in residuals[1:]:
    r_norms.append(np.linalg.norm(r))

In [ ]:
# Print & Plot results
plt.figure(figsize=(10, 4))

# Data
n_conv = len(a_norm_errors)
iterations = range(n_conv)
plt.plot(iterations, np.abs(a_norm_errors), label=f"a-norm")
plt.plot(iterations, np.abs(r_norms), label=f"residuals")

# Grid and Scale
plt.yscale('log')
plt.grid(True)
plt.xlim([0, 1000])
# plt.ylim([-1, 1])
tick_width = 50
plt.xticks([tick_width*i for i in range(int(1050/tick_width))])

# Annotations
plt.xlabel('Iteration')
plt.ylabel(r'A-norm & residual$')
plt.title(r'Convergence of Conjugate Gradient Method')
plt.legend()

# Save and Plot
filename = "cg_convergence.png"
plt.savefig(filename, dpi=300)
plt.show()